# Semantic Kernel s OpenBnB MCP poslužiteljem Integracijom

Ova bilježnica pokazuje kako koristiti Semantic Kernel s stvarnim OpenBnB MCP poslužiteljem za pretraživanje stvarnih Airbnb smještaja pomoću MCPStdioPlugin. Za pristup LLM-u koristi Microsoft Foundry. Za postavljanje svojih varijabli okoline, možete pratiti [Setup Lesson ](/00-course-setup/README.md)


## Uvezi potrebne pakete


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Stvaranje MCP veze dodatka

Povezat ćemo se na [OpenBnB MCP poslužitelj](https://github.com/openbnb-org/mcp-server-airbnb) koristeći MCPStdioPlugin. Ovaj poslužitelj pruža funkcionalnost pretraživanja Airbnb-a putem @openbnb/mcp-server-airbnb paketa.


## Kreiranje klijenta

U ovom primjeru koristit ćemo Microsoft Foundry za pristup našem LLM-u. Provjerite jesu li vaše varijable okoline pravilno postavljene.


## Konfiguracija okoline

Konfigurirajte postavke Azure OpenAI. Pobrinite se da imate postavljene sljedeće varijable okoline:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Razumijevanje OpenBnB MCP integracije

Ova bilježnica se povezuje s **pravim OpenBnB MCP serverom** koji pruža stvarnu funkcionalnost pretraživanja na Airbnbu.

### Kako to funkcionira:

1. **MCPStdioPlugin**: Koristi standardnu komunikaciju ulaza/izlaza s MCP serverom  
2. **Pravi NPM paket**: Preuzima i pokreće `@openbnb/mcp-server-airbnb` putem npx-a  
3. **Stvarni podaci**: Vraća stvarne podatke o nekretninama s njihovih Airbnb API-ja  
4. **Otkrivanje funkcija**: Agent automatski otkriva dostupne funkcije s MCP servera  

### Dostupne funkcije:

OpenBnB MCP server obično pruža:  
- **search_listings** - Pretraživanje Airbnb nekretnina po lokaciji i kriterijima  
- **get_listing_details** - Dobivanje detaljnih informacija o određenim nekretninama  
- **check_availability** - Provjera dostupnosti za određene datume  
- **get_reviews** - Dohvat ocjena za nekretnine  
- **get_host_info** - Dobivanje informacija o domaćinima nekretnina  

### Preduvjeti:

- **Node.js** instaliran na vašem sustavu  
- **Internet veza** za preuzimanje MCP server paketa  
- **NPX** dostupan (dolazi s Node.js-om)  

### Testiranje veze:

Možete ručno testirati MCP server izvođenjem:  
```bash
npx -y @openbnb/mcp-server-airbnb
```
  
Ovo će preuzeti i pokrenuti OpenBnB MCP server, s kojim se Semantic Kernel tada povezuje za stvarne podatke s Airbnb-a.


## Pokretanje Agenta s OpenBnB MCP Serverom

Sada ćemo pokrenuti AI Agenta koji se povezuje na OpenBnB MCP server kako bi pretražio stvarni Airbnb smještaj u Stockholmu za 2 odrasle osobe i 1 dijete. Slobodno promijenite popis `user_inputs` kako biste prilagodili kriterije pretraživanja.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Sažetak
Čestitamo! Uspješno ste izgradili AI agenta koji se integrira s pretragom stvarnog smještaja koristeći Model Context Protocol (MCP):

## Korištene tehnologije:
- Semantic Kernel - Za izradu inteligentnih agenata s Azure OpenAI
- Microsoft Foundry - Za LLM mogućnosti i chat dovršavanje
- MCP (Model Context Protocol) - Za standardiziranu integraciju alata
- OpenBnB MCP Server - Za stvarnu funkcionalnost pretraživanja Airbnb-a
- Node.js/NPX - Za pokretanje vanjskog MCP servera

## Što ste naučili:
- MCP integracija: Povezivanje Semantic Kernel agenata s vanjskim MCP serverima
- Pristup podacima u stvarnom vremenu: Pretraživanje stvarnih Airbnb nekretnina putem live API-ja
- Komunikacija protokolom: Korištenje stdio komunikacije između agenta i MCP servera
- Otkrivanje funkcija: Automatsko otkrivanje dostupnih funkcija s MCP servera
- Streaming odgovora: Snimanje i zapisivanje funkcijskih poziva u stvarnom vremenu
- Prikaz HTML-a: Formatiranje odgovora agenta s stiliziranim tablicama i interaktivnim prikazima

## Sljedeći koraci:
- Integrirati dodatne MCP servere (vremenska prognoza, letovi, restorani)
- Izgraditi sustav s više agenata koji kombinira MCP i A2A protokole
- Kreirati prilagođene MCP servere za vlastite izvore podataka
- Implementirati trajnu memoriju razgovora kroz sesije
- Deployati agenta u Azure Functions s orkestracijom MCP servera
- Dodati korisničku autentifikaciju i mogućnosti rezervacija


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Odricanje od odgovornosti**:
Ovaj dokument je preveden pomoću AI prevoditeljskog servisa [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, molimo imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku smatra se službenim i autoritativnim izvorom. Za važne informacije preporučuje se profesionalni ljudski prijevod. Ne snosimo odgovornost za bilo kakva nesporazuma ili pogrešna tumačenja proizašla iz korištenja ovog prijevoda.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
